In [22]:
import h5py
import json
import numpy as np
import tqdm

# load dataset
data_path = "/home/maelic/Documents/PhD/Datasets/VisualGenome/original_annotations/val.h5"
data = h5py.File(data_path, 'r')

# load curated dataset
curated_path = "/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/datasets/VG150/curated/VG-SGG-with-attri.h5"
curated = h5py.File(curated_path, 'r')
curated_json_dict = json.load(open("/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/datasets/VG150/curated/VG-SGG-dicts-with-attri.json"))

image_data = json.load(open("/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/datasets/vg/image_data.json"))

In [2]:
print(data.keys())

<KeysViewHDF5 ['attributes_per_object', 'image_ids', 'image_paths', 'object_attributes', 'object_boxes', 'object_ids', 'object_names', 'objects_per_image', 'relationship_ids', 'relationship_objects', 'relationship_predicates', 'relationship_subjects', 'relationships_per_image']>


In [23]:
print(curated.keys())

<KeysViewHDF5 ['attributes', 'boxes_1024', 'boxes_512', 'img_to_first_box', 'img_to_first_rel', 'img_to_last_box', 'img_to_last_rel', 'labels', 'predicates', 'relationships', 'split', 'split_rel']>


In [ ]:
print(list(curated['split_rel']).index(0))

indices = [i for i, x in enumerate(list(curated['split_rel'])) if x == 2]
print(indices)

In [ ]:
def generating_files():
    splits = {'train': 0, 'val': 1, 'test': 2}
    for split in ['train', 'val', 'test']:
      final_image_ids = []
      object_ids = []
      object_names = []
      object_boxes = []
      objects_per_image = []
      relationship_ids = []
      relationship_subjects = []
      relationship_predicates = []
      relationship_objects = []
      relationships_per_image = []
      attributes_per_object = []
      object_attributes = []

      indices = [i for i, x in enumerate(list(curated['split_rel'])) if x == splits[split]]

      for image_index in tqdm(indices):
        object_id = []
        object_name = []
        object_box = []

        image_id = image_data[indices]['image_id']

        # Get the number of objects in the image.
        ob_start = curated['img_to_first_box'][image_index]
        ob_end = curated['img_to_last_box'][image_index]
        num_objects = ob_end - ob_start + 1

        for o in range(ob_start, ob_end + 1):
          object_id.append(o)
          object_name.append(curated['labels'][o])
          object_box = curated['object_boxes'][image_index, :num_objects]

        if num_objects == 0:
          continue

        # Get the number of relationships in the image.
        num_relationships = curated['relationships_per_image'][image_index]
        if num_relationships == 0:
          continue

        # Get the object ids, names, and boxes.
        object_id = curated['object_ids'][image_index, :num_objects]
        object_name = curated['object_names'][image_index, :num_objects]
        object_box = curated['object_boxes'][image_index, :num_objects]

        # Get the relationship ids, subjects, predicates, and objects.
        relationship_id = curated['relationship_ids'][image_index, :num_relationships]
        relationship_subject = curated['relationship_subjects'][image_index, :num_relationships]
        relationship_predicate = curated['relationship_predicates'][image_index, :num_relationships]
        relationship_object = curated['relationship_objects'][image_index, :num_relationships]

        # Get the number of attributes per object.
        num_attributes_per_object = curated['attributes_per_object'][image_index, :num_objects]

        # Get the object attributes.
        object_attribute = []
        for j in range(num_objects):
          object_attribute.append(curated['object_attributes'][image_index, j, :num_attributes_per_object[j]])

        # Add the data to the lists.
        final_image_ids.append(image_id)
        object_ids.append(object_id)
        object_names.append(object_name)
        object_boxes.append(object_box)
        objects_per_image.append(num_objects)
        relationship_ids.append(relationship_id)
        relationship_subjects


    numpy_arrays[split] = {
      'image_ids': np.asarray(final_image_ids),
      'object_ids': np.asarray(object_ids),
      'object_names': np.asarray(object_names),
      'object_boxes': np.asarray(object_boxes),
      'objects_per_image': np.asarray(objects_per_image),
      'relationship_ids': np.asarray(relationship_ids),
      'relationship_subjects': np.asarray(relationship_subjects),
      'relationship_predicates': np.asarray(relationship_predicates),
      'relationship_objects': np.asarray(relationship_objects),
      'relationships_per_image': np.asarray(relationships_per_image),
      'attributes_per_object': np.asarray(attributes_per_object),
      'object_attributes': np.asarray(object_attributes),
    }